# Kerasのサンプル
chainerと比べてTitan Blackの環境で4倍遅いことがわかった．  
chainer 5s/iter  
Keras 20s/iter  
Kerasは感覚をつかむのに良いと思ったが，MNISTくらいのデータ・セットしか遅くて使えないか？  
GPUをもっと良いものになれば，使えるか？  
とりあえず今はChainerで行く  
KerasではGPUメモリ消費も非常に多く，以下のプログラムではGPU 5GB使っている

In [1]:
'''Train a simple deep CNN on the CIFAR10 small images dataset.

GPU run command with Theano backend (with TensorFlow, the GPU is automatically used):
    THEANO_FLAGS=mode=FAST_RUN,device=gpu,floatX=float32 python cifar10_cnn.py

It gets down to 0.65 test logloss in 25 epochs, and down to 0.55 after 50 epochs.
(it's still underfitting at that point, though).
'''

from __future__ import print_function
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.layers import BatchNormalization
from keras.optimizers import SGD
from keras.utils import np_utils


Using TensorFlow backend.


In [2]:
import tensorflow as tf
tf.Session(config=tf.ConfigProto(log_device_placement=True))

In [3]:
batch_size = 128
nb_classes = 10
nb_epoch = 300

# input image dimensions
img_rows, img_cols = 32, 32
# The CIFAR10 images are RGB.
img_channels = 3

# The data, shuffled and split between train and test sets:
(X_train, y_train), (X_test, y_test) = cifar10.load_data()
print('X_train shape:', X_train.shape)
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

# Convert class vectors to binary class matrices.
Y_train = np_utils.to_categorical(y_train, nb_classes)
Y_test = np_utils.to_categorical(y_test, nb_classes)

X_train shape: (50000, 3, 32, 32)
50000 train samples
10000 test samples


# Network

In [4]:
model = Sequential()

model.add(Convolution2D(32, 3, 3, input_shape=X_train.shape[1:]))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Convolution2D(64, 3, 3))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Convolution2D(64, 3, 3))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(1000, input_dim=576))
model.add(Activation('relu'))
model.add(Dense(nb_classes))
model.add(Activation('softmax'))

# Run

In [6]:
# Let's train the model
sgd = SGD(lr=0.1, decay=5e-4, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255

print('Not using data augmentation.')
model.fit(X_train, Y_train,
          batch_size=batch_size,
          nb_epoch=nb_epoch,
          validation_data=(X_test, Y_test),
          shuffle=True)
score = model.evaluate(X_test, Y_test, verbose=0)
print('Test score:', score[0])
print('Test accuracy:', score[1])

Not using data augmentation.
Train on 50000 samples, validate on 10000 samples
Epoch 1/300
 1280/50000 [..............................] - ETA: 52s - loss: 2.6427 - acc: 0.1195

KeyboardInterrupt: 

In [6]:
from keras.utils.visualize_util import plot
plot(model, to_file='model.png')

InvocationException: Program terminated with status: 1. stderr follows: Error: /tmp/komatsu/tmplr4Vha: syntax error in line 3 near ':'


In [9]:
import pydot